In [ ]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn import preprocessing
import numpy as np


# Import the data

In [2]:
data = pd.read_csv('train.csv')

# Cleaning the data set


In [3]:
data.rename(columns={'Niños':'Kids'},inplace=True)

In [4]:
data['Edad'].fillna(value=data['Edad'].median(),inplace=True)

In [5]:
data['Pais'].fillna(method='ffill',inplace=True)

In [6]:
data['Kids'].fillna(method='ffill',inplace=True)

# Removing categorical labels


In [7]:
le = preprocessing.LabelEncoder()
data['Genero']=le.fit_transform(data['Genero'])

In [8]:
data['Acomodacion']=le.fit_transform(data['Acomodacion'])

In [9]:
data['Pais']=le.fit_transform(data['Pais'])

In [10]:
data = data.astype({"Edad":'int', "Kids":'int'})

In [11]:
acomodacion=data.Acomodacion

In [12]:
data=data.drop(['Id','Acomodacion','Dias','Pais'],axis=1)

# Split the data

In [13]:
X_train,X_test,Y_train,Y_test = train_test_split(data,acomodacion,test_size=0.2,random_state = 0)

# Input function

In [14]:
def input_fn(features, labels, training=True, batch_size=126):     

    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
      
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

In [15]:
x_cols = []
for key in X_train.keys():
   x_cols.append(tf.feature_column.numeric_column(key=key))


Instantiate the predictor

In [16]:
classifier = tf.estimator.DNNClassifier(
    feature_columns = x_cols,
    hidden_units = [30,10],
    n_classes = 2
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmptvo7kmx4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Training and evaluete the model

In [ ]:
classifier.train(
    input_fn=lambda:input_fn(X_train,Y_train, training=True),
    steps=5000
)

In [18]:
eval = classifier.evaluate(
    input_fn=lambda:input_fn(X_test,Y_test, training=False)
)
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-02-22T23:36:49Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmptvo7kmx4/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.58344s
INFO:tensorflow:Finished evaluation at 2021-02-22-23:36:49
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.5704154, accuracy_baseline = 0.5704154, auc = 0.4838765, auc_precision_recall = 0.61946696, average_loss = 0.6832276, global_step = 5000, label/mean = 0.5704154, loss = 0.6833059, precision = 0.5704154, prediction/mean = 0.5636645, recall = 1.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmptvo7kmx4/model.ckpt-5000

Test set accuracy: 0.570



### La precision del modelo es demasiado baja